In [1]:
from data.process import *

2024-07-28 08:40:10.394313: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-28 08:40:10.419701: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-28 08:40:10.437781: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-28 08:40:10.437816: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-28 08:40:10.458705: I tensorflow/core/platform/cpu_feature_gua

In [2]:
train_df, dev_df, test_df = retrieve_train_dev_test_for_catboost()
X_train, Y_train = split_input_output(train_df)
X_dev, Y_dev = split_input_output(dev_df)

In [3]:
feature_columns = X_train.columns.values
print(feature_columns)

['Gender' 'Age' 'Driving_License' 'Region_Code' 'Previously_Insured'
 'Vehicle_Age' 'Vehicle_Damage' 'Annual_Premium' 'Policy_Sales_Channel'
 'Vintage']


In [4]:
X_train.dtypes

Gender                  object
Age                      int64
Driving_License          int64
Region_Code              int64
Previously_Insured       int64
Vehicle_Age             object
Vehicle_Damage          object
Annual_Premium           int64
Policy_Sales_Channel     int64
Vintage                  int64
dtype: object

In [5]:
import catboost as cb
import catboost.datasets as cbd
import catboost.utils as cbu
import numpy as np
from model.catboost_opt import *
import optuna
import pprint

In [6]:
cat_features = [i for i in range(X_train.shape[1])]
print(cat_features)
print(X_train.columns.values)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
['Gender' 'Age' 'Driving_License' 'Region_Code' 'Previously_Insured'
 'Vehicle_Age' 'Vehicle_Damage' 'Annual_Premium' 'Policy_Sales_Channel'
 'Vintage']


In [7]:
dataset = cb.Pool(X_train, label=Y_train, cat_features=cat_features)

In [8]:
tuned_params = {
    **const_params,
    # tuned params
    'learning_rate': 0.15,
    'depth': 9,
    'l2_leaf_reg': 0.005010366709600069,
    'bagging_temperature': 0.5087874319771577,
    'min_data_in_leaf': 186,
    'leaf_estimation_iterations': 10,
    'eval_metric': 'AUC', 
    'custom_metric': ['AUC'],

    'n_estimators': 3000,
    'early_stopping_rounds': 200,
        
    # --------------- Golden features -------------
    # 'per_float_feature_quantization': ['4:border_count=1024', '6:border_count=1024']
}

In [9]:
# tuned_params = {
#     **const_params,
#     'learning_rate': 0.1,
#     'depth': 8,
#     'bagging_temperature': 0.8,
#     'l2_leaf_reg': 0.00255,
#     'random_strength': 0,
#     'border_count': 32,
#     'n_estimators': 5000,
#     'early_stopping_rounds': 200,
#     'leaf_estimation_iterations': 5,
#     # overriding
#     'eval_metric': 'AUC', 
#     'custom_metric': ['AUC'],

#     # https://catboost.ai/en/docs/concepts/parameter-tuning
#     # --------------- Accuracy -------------
#     # 128 splits are enough for many datasets. However, try to set the value of this parameter to 254 when training on GPU if the best possible quality is required.
#     'border_count': 254,

#     # --------------- Golden features -------------
#     'per_float_feature_quantization': ['4:border_count=1024', '6:border_count=1024']

# }

In [10]:
# can scale_pos_weight boost up the performance for imbalanced dataset?
scale_pos_weight = np.sum(Y_train == 0) / np.sum(Y_train == 1)

In [11]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)


In [12]:
# for fold, (train_idx, valid_idx) in enumerate(skf.split(train_df, train_df['Response'])):
#     print(f'### Fold {fold+1} Training ###')

#     X_train = train_df.iloc[train_idx][feature_columns]
#     y_train = train_df.iloc[train_idx]['Response']
#     X_valid = train_df.iloc[valid_idx][feature_columns]
#     y_valid = train_df.iloc[valid_idx]['Response']
#     X_test = test_df[feature_columns]

#     X_train_pool = cb.Pool(X_train, y_train, cat_features=cat_features)
#     X_valid_pool = cb.Pool(X_valid, y_valid, cat_features=cat_features)
#     X_test_pool = cb.Pool(X_test, cat_features=cat_features)

#     model = cb.CatBoostClassifier(**tuned_params, cat_features=cat_features, scale_pos_weight=scale_pos_weight)
#     model.fit(X_train_pool, verbose=500, eval_set=X_train_pool)

#     pprint.pprint(model.best_score_)

In [13]:
%%time
model = cb.CatBoostClassifier(**tuned_params, cat_features=cat_features, scale_pos_weight=scale_pos_weight)
model.fit(dataset, eval_set=(X_dev, Y_dev), verbose_eval=100)

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.8740111	best: 0.8740111 (0)	total: 1.2s	remaining: 1h 1s
100:	test: 0.8929405	best: 0.8929405 (100)	total: 1m 39s	remaining: 47m 22s
200:	test: 0.8940838	best: 0.8940840 (199)	total: 3m 5s	remaining: 43m
300:	test: 0.8945602	best: 0.8945602 (300)	total: 4m 27s	remaining: 39m 56s
400:	test: 0.8947477	best: 0.8947608 (376)	total: 5m 51s	remaining: 37m 56s
500:	test: 0.8947946	best: 0.8948061 (487)	total: 7m 14s	remaining: 36m 5s
600:	test: 0.8948357	best: 0.8948490 (583)	total: 8m 35s	remaining: 34m 16s
700:	test: 0.8948260	best: 0.8948490 (583)	total: 9m 57s	remaining: 32m 38s
800:	test: 0.8948473	best: 0.8948791 (742)	total: 11m 21s	remaining: 31m 9s
900:	test: 0.8948410	best: 0.8948791 (742)	total: 12m 43s	remaining: 29m 37s
bestTest = 0.8948791027
bestIteration = 742
Shrink model to first 743 iterations.
CPU times: user 15min 44s, sys: 1min 2s, total: 16min 46s
Wall time: 13min 32s


In [14]:
pprint.pprint(model.best_iteration_)
pprint.pprint(model.best_score_)

742
{'learn': {'Logloss': 0.3711196014855591},
 'validation': {'AUC': 0.8948791027069092, 'Logloss': 0.3787688223179413}}


In [15]:
# model.save_model('catboost_model.json', format='json', pool=dataset)

In [16]:

# model.save_model('catboost_model_categorical.cbm')

In [17]:
# test_pool = cb.Pool(test_df, cat_features=X_train.columns.values)
# cb_output = model.predict(test_df, prediction_type='Probability', ntree_end=model.get_best_iteration())

In [18]:
# Two-dimensional numpy.ndarray of shape (number_of_objects, number_of_classes) with the probability for every class for each object.
# print(cb_output)

In [19]:
# since it is binary classification, we only need the second column
# cb_output = cb_output[:, 1]